<a href="https://colab.research.google.com/github/shreyaskm51/Human-Activity-Recognition-with-CNN-LSTM/blob/main/ai-har.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision opencv-python scikit-learn

import os
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from google.colab import files
from IPython.display import HTML, display
from base64 import b64encode

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

classes = ["walking", "running", "falling"]

for cls in classes:
    os.makedirs(f"data/{cls}", exist_ok=True)
    os.makedirs(f"features/{cls}", exist_ok=True)

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

resnet = models.resnet18(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1]).to(device)
resnet.eval()

print(" Upload your videos now. They will be sorted into 'data/{class}' folders automatically based on filename.")

uploaded = files.upload()

print("\n Moving uploaded files into folders...")
for filename in uploaded.keys():
    lower_name = filename.lower()
    if "walk" in lower_name:
        out_path = f"data/walking/{filename}"
    elif "run" in lower_name:
        out_path = f"data/running/{filename}"
    elif "fall" in lower_name:
        out_path = f"data/falling/{filename}"
    else:
        out_path = f"data/walking/{filename}"
    os.rename(filename, out_path)
    print(f" {filename} → {out_path}")

print("\n Upload & sorting done! Run next cell to extract features.")

def extract_features():
    for cls in classes:
        in_dir = f"data/{cls}"
        out_dir = f"features/{cls}"
        os.makedirs(out_dir, exist_ok=True)

        video_files = glob.glob(f"{in_dir}/*.mp4")
        if not video_files:
            print(f" No videos found in '{in_dir}', skipping...")
            continue

        for vid_path in video_files:
            npy_path = os.path.join(out_dir, os.path.splitext(os.path.basename(vid_path))[0] + ".npy")
            if os.path.exists(npy_path):
                print(f"ℹ Features already extracted for {vid_path}, skipping.")
                continue

            print(f" Extracting features from {vid_path} ...")
            cap = cv2.VideoCapture(vid_path)
            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = transform(frame).unsqueeze(0).to(device)
                with torch.no_grad():
                    feat = resnet(frame).cpu().numpy().reshape(-1)
                frames.append(feat)
            cap.release()
            if frames:
                feat_arr = np.stack(frames)
                np.save(npy_path, feat_arr)
                print(f" Saved features to {npy_path}")

extract_features()

class FeatureDataset(torch.utils.data.Dataset):
    def __init__(self, files, labels):
        self.files = files
        self.labels = labels
    def __len__(self):
        return len(self.files)
    def __getitem__(self, i):
        x = np.load(self.files[i])
        x = torch.from_numpy(x).float()
        return x, self.labels[i]

def pad_collate(batch):
    features = [item[0] for item in batch]
    labels = torch.tensor([item[1] for item in batch])
    lengths = [f.shape[0] for f in features]
    max_len = max(lengths)
    feat_dim = features[0].shape[1]

    padded_feats = torch.zeros(len(features), max_len, feat_dim)
    for i, f in enumerate(features):
        length = f.shape[0]
        padded_feats[i, :length, :] = f

    return padded_feats, labels

all_files, all_labels = [], []
for idx, cls in enumerate(classes):
    found = glob.glob(f"features/{cls}/*.npy")
    print(f" Found {len(found)} feature files for '{cls}'")
    for f in found:
        all_files.append(f)
        all_labels.append(idx)

if len(all_files) == 0:
    raise ValueError("No features found! Please run feature extraction first.")

min_per_class = min(len([l for l in all_labels if l == cls_idx]) for cls_idx in set(all_labels))
if min_per_class < 2:
    print(" Few samples detected; skipping stratified split.")
    tr_f, val_f, tr_l, val_l = train_test_split(all_files, all_labels, test_size=0.33, random_state=42)
else:
    tr_f, val_f, tr_l, val_l = train_test_split(all_files, all_labels, test_size=0.33, random_state=42, stratify=all_labels)

train_ds = FeatureDataset(tr_f, tr_l)
val_ds = FeatureDataset(val_f, val_l)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True, collate_fn=pad_collate)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, collate_fn=pad_collate)

feat_dim = np.load(all_files[0]).shape[1]

class VideoClassifierLSTM(nn.Module):
    def __init__(self, feat_dim, hidden_dim, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(feat_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):

        _, (hn, _) = self.lstm(x)
        hn = hn.squeeze(0)
        return self.fc(hn)

model = VideoClassifierLSTM(feat_dim, hidden_dim=128, num_classes=len(classes)).to(device)

crit = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=0.001)

MODEL_PATH = "model_lstm.pth"

if os.path.exists(MODEL_PATH):
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    print(f" Loaded saved model from {MODEL_PATH}")
else:
    print("ℹ No saved model found, will train a new one.")

if not os.path.exists(MODEL_PATH):
    for epoch in range(5):
        model.train()
        train_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), torch.tensor(yb).to(device)
            opt.zero_grad()
            out = model(xb)
            loss = crit(out, yb)
            loss.backward()
            opt.step()
            train_loss += loss.item()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), torch.tensor(yb).to(device)
                out = model(xb)
                preds = torch.argmax(out, dim=1)
                correct += (preds == yb).sum().item()
                total += yb.size(0)

        print(f"Epoch {epoch+1} | Train Loss: {train_loss/len(train_loader):.4f} | Val Acc: {correct/total:.2f}")

    torch.save(model.state_dict(), MODEL_PATH)
    print(f" Model trained and saved to {MODEL_PATH}")
else:
    print(" Model already trained, skipping training step.")

def show_video(video_path):
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"<video width=400 controls><source src='{data_url}' type='video/mp4'></video>")

def predict_video(video_path, show_vid=False):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_tensor = transform(frame_rgb).unsqueeze(0).to(device)
        with torch.no_grad():
            feat = resnet(frame_tensor).cpu().numpy().reshape(-1)
        frames.append(feat)
    cap.release()

    if frames:
        feat_arr = np.stack(frames)
        x = torch.from_numpy(feat_arr).unsqueeze(0).float().to(device)
        model.eval()
        with torch.no_grad():
            pred = torch.argmax(model(x), dim=1).item()
        if show_vid:
            display(show_video(video_path))
        return classes[pred]
    else:
        return None

uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print(f"Uploaded video: {video_path}")
result = predict_video(video_path, show_vid=True)
print("Prediction:", result)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 177MB/s]


 Upload your videos now. They will be sorted into 'data/{class}' folders automatically based on filename.


Saving A man running - side view - Animation reference.mp4 to A man running - side view - Animation reference.mp4
Saving Falling down stairs.mp4 to Falling down stairs.mp4
Saving Falling on Ice 🧊 #fall #funny #fail #ice #slide #winter #santa.mp4 to Falling on Ice 🧊 #fall #funny #fail #ice #slide #winter #santa.mp4
Saving falling.mp4 to falling.mp4
Saving Indian fastest runner🏃💨 _ Speed _ #shorts.mp4 to Indian fastest runner🏃💨 _ Speed _ #shorts.mp4
Saving Man walk cycle_ happy walking side view reference- Animation Reference Videos.mp4 to Man walk cycle_ happy walking side view reference- Animation Reference Videos.mp4
Saving Man walking on road side view 🔥 _ Video for kinemaster video editing 🤠.mp4 to Man walking on road side view 🔥 _ Video for kinemaster video editing 🤠.mp4
Saving Running  Video _ Free HD Video - No Copyright..mp4 to Running  Video _ Free HD Video - No Copyright..mp4
Saving walking.mp4 to walking.mp4

 Moving uploaded files into folders...
 A man running - side view -

/tmp/ipython-input-3271297658.py:176: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xb, yb = xb.to(device), torch.tensor(yb).to(device)
/tmp/ipython-input-3271297658.py:188: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xb, yb = xb.to(device), torch.tensor(yb).to(device)


Epoch 1 | Train Loss: 1.0764 | Val Acc: 0.00
Epoch 2 | Train Loss: 1.0024 | Val Acc: 0.00
Epoch 3 | Train Loss: 0.9620 | Val Acc: 0.00
Epoch 4 | Train Loss: 0.9396 | Val Acc: 0.00
Epoch 5 | Train Loss: 0.9266 | Val Acc: 0.00
 Model trained and saved to model_lstm.pth


Saving Male Walk Cycle Animation Reference _ Front & Side Views & Realtime playback speed.mp4 to Male Walk Cycle Animation Reference _ Front & Side Views & Realtime playback speed.mp4
Uploaded video: Male Walk Cycle Animation Reference _ Front & Side Views & Realtime playback speed.mp4


Prediction: running
